# 무신사 상품 리뷰 크롤링(crawling)

## 환경설정

In [1]:
#필요하면 설치하기
#!pip install webdriver-manager
#from selenium import webdriver
#from webdriver_manager.chrome import ChromeDriverManager
#driver = webdriver.Chrome(ChromeDriverManager().install())
#!pip install selenium

In [3]:
import warnings
warnings.filterwarnings("ignore")
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import re
import time
import pandas as pd

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from html_table_parser import parser_functions as parser

 

In [3]:
#함수정의:검색어 조건에 따른 url생성
#무신사 스탠다드->후기순->벨트 및 양말 등 잡화제외 1위 상품
#def musinsa_searching(word):
#  url = "https://www.musinsa.com/app/goods/1149328" 
#  return url

In [79]:
#driver = webdriver.Chrome('chromedriver.exe')
#위치: C:\Users\rlawndms\boaz_삭제절대금지
#executable_path='C:/Users/rlawndms/boaz_삭제절대금지/chromedriver.exe'
#후드
options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-logging"])
driver = webdriver.Chrome(options=options)
driver.get('https://www.musinsa.com/app/goods/2326935')
time.sleep(5)

In [5]:
# 할인쿠폰 창 없애기->그냥 수동으로 했음

## 함수정의 
* 리뷰크롤링
* 페이지이동

In [80]:
#user, gender,height,weight,item, size,content, evaluation(size_eval,bright_eval,color_eval,thick_eval)
user_list = []
gender_list =[]
height_list = []
weight_list =[]
item_list = []
size_list = []
star_list= []
content_list = []
size_eval_list =[]
bright_eval_list =[]
color_eval_list =[]
thick_eval_list =[]

#함수정의
def get_content(driver):
    #함수안에 html, soup 넣어놔야 페이지 넘어가서 바르게 긁어옴, 밖에 빼놓으면 첫페이지만 여러번 긁어진다.
    html = driver.page_source
    soup = BeautifulSoup(html,'lxml')
    for i in range(10):
    
    #profile(gender,height,weight)
    #<p class="review-profile__body_information">남성, 177cm, 85kg</p>
        profile_before = soup.find_all('p','review-profile__body_information')
        profile_after = profile_before[i].text.split(',')
        try:
            gender = profile_after[0]
            height = profile_after[1]
            weight = profile_after[2]
        except:
            gender = ''
            height = ''
            weight = ''
#user :<p class="review-profile__name">LV 2 뉴비_95f88e16</p>           
#item: #/<a href="https://www.musinsa.com/app/goods/1231416/0" class="review-goods-information__name">테이퍼드 히든 밴딩 크롭 슬랙스 [더스티 베이지]</a>
#size :<span class="review-goods-information__option">
#content

        try:
            user = soup.find_all('p','review-profile__name')[i].text
            item = soup.find_all('a','review-goods-information__name')[i].text
            # '/n' 없애고 추출하기
            size = soup.find_all('span', 'review-goods-information__option')[i].text.strip().replace('\n','')
            content = soup.find_all('div','review-contents__text')[i].text
        except:
            user = ''
            item = ''
            size = ''
            content = ''
            
        #star
#->별 5개일때:<span class="review-list__rating__active" style="width: 100%"></span>
#->별 4개일때:<span class="review-list__rating__active" style="width: 80%"></span>
        stars = driver.find_elements_by_xpath('//*[@id="reviewListFragment"]/div['+str(i+1)+']/div[3]/span/span/span')
        try:
            for j in stars:
                a =j.get_attribute('style')
                if a[7:9]=='20':
                    star = 1
                elif a[7:9]=='40':
                    star = 2
                elif a[7:9]=='60':
                    star = 3
                elif a[7:9]=='80':
                    star = 4
                else:
                    star = 5
        except:
            star = ''
      

    #evaluation
        evaluation = soup.find_all('div', 'review-evaluation')
        try:
            size_eval = evaluation[i].find_all('span')[0].text
            bright_eval = evaluation[i].find_all('span')[1].text
            color_eval = evaluation[i].find_all('span')[2].text
            thick_eval = evaluation[i].find_all('span')[3].text
        except:
            size_eval = ''
            bright_eval = ''
            color_eval = ''
            thick_eval = ''

#user,gender,height,weight,item, size,content,star,size_eval,bright_eval,color_eval,thick_eval
        
        user_list.append(user)
        gender_list.append(gender)
        height_list.append(height)
        weight_list.append(weight)
        item_list.append(item)
        size_list.append(size)
        content_list.append(content)
        star_list.append(star)
        size_eval_list.append(size_eval)
        bright_eval_list.append(bright_eval)
        color_eval_list.append(color_eval)
        thick_eval_list.append(thick_eval)


In [81]:
#버튼 누르기 함수정의
def move_next(driver):    
    for i in range(4):
        get_content(driver)
        #페이지 2,3,4,5 넘어가기
        driver.find_element_by_css_selector('#reviewListFragment > div.nslist_bottom > div.pagination.textRight > div > a:nth-child(' + 
                                            str(int(4) + int(i)) + ')').send_keys(Keys.ENTER)
        time.sleep(2)
    get_content(driver)
    
#그다음 화살표'>'버튼누르기: (6,7,8...)있는 페이지로 넘어가기   
def move_arrow(driver):
    driver.find_element_by_css_selector('#reviewListFragment > div.nslist_bottom > div.pagination.textRight > div > a.fa.fa-angle-right.paging-btn.btn.next').send_keys(Keys.ENTER)
    

## 크롤링 시작(Start)

In [82]:
# 1->50개, 2->100개, 20->1000개, 40->2000개(1537개 긁어짐 왤까 코드 수정필요할 듯)
#200->10000개(예상) 16:20시작, #103페이지부터 안돌아감
#숫자 조정해주기
html = driver.page_source
soup = BeautifulSoup(html,'html.parser')
figure = soup.find('table',{'class':'table_th_grey'})
p = parser.make2d(figure)
figure_df = pd.DataFrame(data = p[1:],columns = p[0])
figure_df.drop([0,1],inplace = True)

for i in range(1):
     try:
        move_next(driver)
        move_arrow(driver)
    #move_next(driver)
     except:
        time.sleep(2)
    #move_next(driver)
    

df = pd.DataFrame({'user':user_list, 
                   'gender':gender_list,
                   'height':height_list ,
                   'weight':weight_list,
                   'item':item_list, 
                   'size':size_list,
                   'star':star_list,
                   'content':content_list,
                   'size_eval':size_eval_list,
                   'bright_eval':bright_eval_list,
                   'color_eval':color_eval_list,
                   'thick_eval':thick_eval_list})


In [83]:
figure_df

,cm,총장,어깨너비,가슴단면,소매길이
2,WS(논기모),60,52,53,58
3,S(논기모),63,55,56,60
4,M(논기모),67,58,59,62
5,L(논기모),69,60,61,63
6,XL(논기모),71,62,63,64


In [85]:
merge_df = pd.merge(df,figure_df,how = 'left',left_on = 'size',right_on = 'cm')
merge_df.head()

,user,gender,height,weight,item,size,star,content,size_eval,bright_eval,color_eval,thick_eval,cm,총장,어깨너비,가슴단면,소매길이
0,LV 3 뉴비_b01a76d03b20,남성,178cm,75kg,(22FW) 2 TONE ARCH HOODIE GRAY,XL(논기모),5,적당히 이쁨가격좀 있는데 세일할때 사서 행복시보리들 짱짱해서 좋고 허리 시보리 짱짱...,보통이에요,보통이에요,보통이에요,보통이에요,XL(논기모),71,62,63,64
1,LV 3 크리스리78,남성,175cm,60kg,(22FW) 2 TONE ARCH HOODIE GRAY,M(논기모),5,저렴한 가격에 잘 산것 같아요 커플티로 매우만족합니다 단 냄새는 조금 나네요 날라가...,커요,보통이에요,선명해요,보통이에요,M(논기모),67,58,59,62
2,LV 5 브론즈_763c9863216,여성,160cm,52kg,(22FW) 2 TONE ARCH HOODIE GRAY,S(논기모),5,딱맞네요.한치수커도 괜찮을것같은데 길이감이 조금 아쉽네요.,보통이에요,보통이에요,보통이에요,보통이에요,S(논기모),63,55,56,60
3,LV 3 반뒤뒤,남성,168cm,62kg,(22FW) 2 TONE ARCH HOODIE BLACK,M,5,살까 말까 많이 고민했는데 사는 사람이 많은 걸 보니 사기로 했어요.기대보다 너ㅓ무...,보통이에요,보통이에요,보통이에요,보통이에요,NaN,NaN,NaN,NaN,NaN
4,LV 4 파스타먹기,남성,170cm,68kg,(22FW) 2 TONE ARCH HOODIE GRAY,L(논기모),4,사이즈도 적당히 오버핏이고 이뻐요 다만 주변에 너무 많이 보이는게 흠,보통이에요,보통이에요,보통이에요,보통이에요,L(논기모),69,60,61,63


In [4]:
df = pd.read_csv('[221116]Hood1_Test.csv')
df.head()

,Unnamed: 0,user,gender,height,weight,item,size,star,content,size_eval,bright_eval,color_eval,thick_eval,cm,총장,어깨너비,가슴단면,소매길이
0,0,LV 2 재군님,남성,169cm,72kg,(22FW) 2 TONE ARCH HOODIE GRAY,L(논기모),5,우선 배송이 빨라서 좋았습니다. 상품 받았는데 사진에서 봤던거랑 똑같났습니다. 디자...,보통이에요,보통이에요,선명해요,보통이에요,L(논기모),69.0,60.0,61.0,63.0
1,1,LV 3 뇨후,여성,175cm,70kg,(22FW) 2 TONE ARCH HOODIE GRAY,L(논기모),5,소매부분이 짱짱해서 제일 맘에 들고 비교적 박시한 스타일의 후드는 아니지만 길이가 길어요,커요,보통이에요,선명해요,보통이에요,L(논기모),69.0,60.0,61.0,63.0
2,2,LV 3 뉴비_38b25b18,남성,170cm,65kg,(22FW) 2 TONE ARCH HOODIE GRAY,L(논기모),5,전역선물로 사줬는데 가장 기본템이라서 좋대요!,보통이에요,보통이에요,보통이에요,보통이에요,L(논기모),69.0,60.0,61.0,63.0
3,3,LV 5 무신쿡,남성,178cm,72kg,(22FW) 2 TONE ARCH HOODIE GRAY,L(논기모),5,"제 스펙 기준 L 사이즈 조금 끼는 느낌. 처음으로 산 예일 브랜드 옷인데, 무난하...",보통이에요,보통이에요,보통이에요,보통이에요,L(논기모),69.0,60.0,61.0,63.0
4,4,LV 3 뉴비_5f8502fbdb87,남성,163cm,51kg,(22FW) 2 TONE ARCH HOODIE BLACK,M,4,예약배송이라 한참 기다리다 받았는데 블프 세일해서 만원 넘게 더 비싼 가격에 사서 ...,보통이에요,보통이에요,보통이에요,보통이에요,NaN,NaN,NaN,NaN,NaN


## 비고
* 시험용 코드 및 중간 과정

In [57]:
html = driver.page_source
soup = BeautifulSoup(html,'html.parser')

figure = soup.find('table',{'class':'table_th_grey'})
p = parser.make2d(figure)
figure_df = pd.DataFrame(data = p[1:],columns = p[0])
figure_df

,cm,총장,어깨너비,가슴단면,소매길이
0,MY,가지고 계신 제품의 실측을 입력해 보세요~!,가지고 계신 제품의 실측을 입력해 보세요~!,가지고 계신 제품의 실측을 입력해 보세요~!,가지고 계신 제품의 실측을 입력해 보세요~!
1,위 구매 내역의 사이즈를 저장하시겠습니까?,위 구매 내역의 사이즈를 저장하시겠습니까?,위 구매 내역의 사이즈를 저장하시겠습니까?,위 구매 내역의 사이즈를 저장하시겠습니까?,위 구매 내역의 사이즈를 저장하시겠습니까?
2,WS(논기모),60,52,53,58
3,S(논기모),63,55,56,60
4,M(논기모),67,58,59,62
5,L(논기모),69,60,61,63
6,XL(논기모),71,62,63,64


In [58]:
driver = webdriver.Chrome()
#해당 상품 링크_
#무신사->상의->후드티셔츠->무신사추천순->top1~5
#yale 후드티
driver.get('https://www.musinsa.com/app/goods/2314616')
time.sleep(3)

html = driver.page_source
soup = BeautifulSoup(html,'html.parser')

figure = soup.find('table',{'class':'table_th_grey'})
p = parser.make2d(figure)
figure_df = pd.DataFrame(data = p[1:],columns = p[0])
figure.drop([0,1],inplace = True)
figure_df


,cm,총장,어깨너비,가슴단면,소매길이
0,MY,가지고 계신 제품의 실측을 입력해 보세요~!,가지고 계신 제품의 실측을 입력해 보세요~!,가지고 계신 제품의 실측을 입력해 보세요~!,가지고 계신 제품의 실측을 입력해 보세요~!
1,위 구매 내역의 사이즈를 저장하시겠습니까?,위 구매 내역의 사이즈를 저장하시겠습니까?,위 구매 내역의 사이즈를 저장하시겠습니까?,위 구매 내역의 사이즈를 저장하시겠습니까?,위 구매 내역의 사이즈를 저장하시겠습니까?
2,S,66,59,65.5,58
3,M,68,60.5,67.5,59
4,L,70,62,69.5,60
5,XL,72,63.5,71.5,61


In [61]:
#0,1행은 공통적으로 my사이즈와 저장하시겠습니까를 물어보는 행이라 삭제
figure_df.drop([0,1],axis =0,inplace = True)
figure_df

,cm,총장,어깨너비,가슴단면,소매길이
2,S,66,59,65.5,58
3,M,68,60.5,67.5,59
4,L,70,62,69.5,60
5,XL,72,63.5,71.5,61


In [29]:
# df생략없이 출력할 때
import numpy as np
np.set_printoptions(threshold=np.inf,linewidth=np.inf)
import pandas as pd
pd.set_option('display.max_columns',10)
# row 생략 없이 출력
pd.set_option('display.max_rows', None)
# col 생략 없이 출력
pd.set_option('display.max_columns', None)

In [ ]:
# 들어가자마자 뜨는 할인쿠폰 창 없애기
#from 박태남
try:
    button = browser.find_element_by_class_name('ab-close-button')
    button.send_keys(Keys.ENTER)
    print("coupon close")
except:
    print("no coupon")
time.sleep(2)